In [3]:
# install: pip install --upgrade arabic-reshaper
import arabic_reshaper
#imports for hash function and qr
import hashlib
import qrcode
# install: pip install python-bidi
from bidi.algorithm import get_display
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import os

In [9]:
import smtplib
import imghdr
from email.message import EmailMessage

In [ ]:
df = pd.read_csv('F:/ipynb/cert/test2.csv')
fontFile = "F:/ipynb/cert/arial.ttf"
imageFile = "F:/ipynb/cert/certificate.jpg"
font = ImageFont.truetype(fontFile,60)
for index,j in df.iterrows():
    reshaped_text = arabic_reshaper.reshape('{}'.format(j['Name']))    # correct its shape
    bidi_text = get_display(reshaped_text)
    #start make hash and qr
    hash_object = hashlib.md5(bidi_text.encode())
    print(hash_object.hexdigest())
    input_data = hash_object
    #Creating an instance of qrcode
    qr = qrcode.QRCode(
        version=1,
        box_size=10,
        border=1)
    qr.add_data(input_data.hexdigest())
    qr.make(fit=True)
    imgQR = qr.make_image(fill='black', back_color='white')
    imgQR.save('F:/ipynb/cert/pics/{}.png'.format(j['Name']))
    new_image = imgQR.resize((140, 140))
    #end make hash and qr
    img = Image.open('F:/ipynb/cert/certificate.jpg')
    draw = ImageDraw.Draw(img)
    draw.text(xy=(525,440),text=bidi_text,fill=(0,0,0),font=font)
    img.paste(new_image, (50,1200))
    img.save('F:/ipynb/cert/pics/{}.jpg'.format(j['Name']))
    #start sending emails
    idsr = arabic_reshaper.reshape('{}'.format(j['Emails']))
    print(idsr)
    Sender_Email = "your@email.com"
    Reciever_Email = arabic_reshaper.reshape('{}'.format(j['Emails']))
    Password = input('Enter your email account password: ')
    newMessage = EmailMessage()                         
    newMessage['Subject'] = "your subject is here - ضع هنا اسم الموضوع" 
    newMessage['From'] = Sender_Email                   
    newMessage['To'] = Reciever_Email                   
    newMessage.set_content('Message content gose here - محتوى نص الرسالة ') 
    with open('F:/ipynb/cert/pics/{}.jpg'.format(j['Name']), 'rb') as f:
        image_data = f.read()
        image_type = imghdr.what(f.name)
        image_name = f.name
    newMessage.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)

    # use 'smtp.gmail.com', 465 with gmail, otherwise use your smtp provider if you are using different provider
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(Sender_Email, Password)              
        smtp.send_message(newMessage)
    #end sending emails